##Textual Data Anonymization and Analysis

-Input: Textual Medical Data

-Approach
**i)** Extract Personal Information (PI)/Entities Using Chat-GPT. This includes replacing these entities with Placeholders
**ii)** Map/Replace each PI to anonymized representations. 
**iii)** Replacing Placeholders with anonymized representations

-Output: Anonymized Text

In [0]:
!pip install openai
import openai
import pandas as pd
import numpy as np
import os
import ast
import matplotlib.pyplot as plt
# import tiktoken
import warnings
import json

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [0]:
#Configure OpenAI connection
openai.api_key = dbutils.secrets.get(scope="openai", key="api-key")
openai.api_base = "https://gsk-ds.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_type = "azure"

In [0]:
class OpenaiAnnotater():
  # openai.api_key = dbutils.secrets.get(scope="openai", key="api-key")
  def __init__(self, engine = "ds-gpt-35", temp = 0):
    self.engine = engine
    self.temp = temp

  def question_answer(self, system_prompt, user_prompt):
    # for prompt in pronmpts:
    messages = [{"role": "system", "content": system_prompt},{"role" : "user", "content" : user_prompt}]
    response = openai.ChatCompletion.create(
      engine = self.engine,
      temperature = self.temp,
      messages=messages,
      #max_tokens=2000
    )
    return response.choices[0].message["content"], response.usage["total_tokens"]
  

In [0]:
annotater  = OpenaiAnnotater()

In [0]:
blob_path = '/dbfs/mnt/medicalaffairs/Deepti_test/'
reportname='InputText_modified_names.xlsx'


In [0]:
!pip install xlsxwriter
# Specify the Excel file name
out_excel_file_name = blob_path+"Input Text Out.xlsx"
# Create an ExcelWriter object


writer = pd.ExcelWriter('tmp_out.xlsx', engine='xlsxwriter') #to write in excel - temporary


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [0]:
#samples for training
df=pd.read_excel(blob_path+reportname)

In [0]:
df.columns

Index(['SlNo', 'Text'], dtype='object')

In [0]:
len(df)

20

###Extract entities and generate a CHAT-GPT response along with Anonymized Text with placeholders

In [0]:
enity_list=["name", "address", "email-id", "phone number", "disease"]

In [0]:
system_prompt= f"Act as an AI expert. You will be given data to extract entities from it. The entities that you need to extract are given in the list ``` {enity_list} ``` . In the absence of any entity specify message Not Available. Once you extract entity -  anonymize the text with the entities listed in the ``` {enity_list} in placeholder <> only`` while preserving the original data's utility for analysis. The anonymization should be performed in a way that makes it difficult to reverse engineer the original entities, ensuring data privacy and confidentiality. Answer in consistent style. Do not add any information which is not present in the data. Cross-check your answer before for accuracy and logical consitency"
df_results = pd.DataFrame()  




In [0]:
for index, row in df.iterrows():
      
      user_prompt = f""" Extract entity for the below text.
                  
                  Text: { row['Text']}
                  Name: <NAME>
                  Address: <ADDRESS>
                  Email Id: <EMAIL>
                  Phone number: <PHONE NUMBER>
                  Disease: <DISEASE>
                  Anonymized Text: <ANONYMIZED TEXT>
                   """
     
      response, tokens = annotater.question_answer(system_prompt, user_prompt)
      #print(response)
      df_results.at[index,'SlNo']= row['SlNo']
      df_results.at[index,'Text']= row['Text']
      df_results.at[index,'Response']= response
     
     
          
#print(system_prompt)
print(df_results.columns)


Index(['SlNo', 'Text', 'Response'], dtype='object')


In [0]:
df_results.head(2)

SlNo  ...                                           Response
0   1.0  ...  Name: Jake Rodriguez\nAddress: 789 Elm Street,...
1   2.0  ...  Name: Emma White\nAddress: 505 Pine Lane, Metr...

[2 rows x 3 columns]

###Pre-process CHAT-GPT response
- Output entities in different columns
- Lemmatize Disease : Example : Allergy or Allergies are same so make them allergy

In [0]:

new_df= pd.DataFrame()
def remove_empty(lst):
    return [x for x in lst if x != [''] and x != ['                  ']]
  
for index, row in df_results.iterrows():
 #split response
      lines = row['Response'].split('\n')
      #print(lines)

      lines = [item for item in lines if item!=[]]
      entity_value_pair = [line.split(':') for line in lines] 
      #print(entity_value_pair)
      entity_value_pair=remove_empty(entity_value_pair)
      print(entity_value_pair)
      #Create dictionary
      
      data = {entity.strip(): value.strip() for entity,value in entity_value_pair}
      data['SlNo']= row['SlNo']
      data['Text'] = row['Text']
      data['Response'] = row['Response']
      
      
      #new_df = new_df.append(data, ignore_index=True) #deprecated
      new_df=pd.concat([new_df, pd.DataFrame([data])], ignore_index=True)

new_df.columns

#Reorder columns
new_df = new_df[['SlNo','Text','Response','Anonymized Text','Name', 'Address', 'Email Id', 'Phone number', 'Disease']]
new_df 

[['Name', ' Jake Rodriguez'], ['Address', ' 789 Elm Street, Riverside'], ['Email Id', ' Not Available'], ['Phone number', ' (555) 123-4567'], ['Disease', ' cold'], ['Anonymized Text', " Hi, I'm <NAME>, you can reach me at <PHONE NUMBER>. Just moved to <ADDRESS>. Trying to beat this <DISEASE> and binge-watching classic movies for comfort."]]
[['Name', ' Emma White'], ['Address', ' 505 Pine Lane, Metroville'], ['Email Id', ' Not Available'], ['Phone number', ' Not Available'], ['Disease', ' flu'], ['Anonymized Text', " This is <NAME> speaking, no calls, please. I'm at <ADDRESS>. Recovering from a <DISEASE>, spending my days sketching in the garden, a peaceful healing process."]]
[['Name', ' Harper David'], ['Address', ' 909 Oakwood Road, Lakeside'], ['Email Id', ' Not Available'], ['Phone number', ' (555) 987-6543'], ['Disease', ' allergies'], ['Anonymized Text', " Hey, it's <NAME> here. Call me on <PHONE NUMBER>. Living at <ADDRESS>. Dealing with <DISEASE> lately, but enjoying long walk

SlNo  ...                           Disease
0    1.0  ...                              cold
1    2.0  ...                               flu
2    3.0  ...                         allergies
3    4.0  ...                     Not Available
4    5.0  ...  Recovering from a recent surgery
5    6.0  ...                     Not Available
6    7.0  ...                          Insomnia
7    8.0  ...                     Not Available
8    9.0  ...                      Minor injury
9   10.0  ...                     Not Available
10  11.0  ...                              cold
11  12.0  ...                               flu
12  13.0  ...                         allergies
13  14.0  ...                     Not Available
14  15.0  ...                     Not Available
15  16.0  ...                     Not Available
16  17.0  ...                     Not Available
17  18.0  ...                     Not Available
18  19.0  ...                              cold
19  20.0  ...                         arthritis

[20 rows x 9 columns]

In [0]:
#Lemmatize Diseaese column for example allergies to allergy etc

# Create WordNetLemmatizer object

lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return lemmatizer.lemmatize(text.lower()) 


new_df['Disease'] = new_df.Disease.apply(lemmatize_text)
new_df.head(2)

SlNo  ... Disease
0   1.0  ...    cold
1   2.0  ...     flu

[2 rows x 9 columns]

###Create Abbreviations for Entities

In [0]:
#Create dictionary of entities Name, Address, EmailId, Phone muber and Disease, 
# For entity Name, give Initial letters as it's value. Example: James Kalra as JK1, Nix Kashyap as NK, but if also Neha Kashyap give as NK2 and soon.Create this unique dictionary
# For entity Address give it as Add1, Add2
# 
#initialize dictionaries

name_dict={}
address_dict={}
email_dict={}
phone_dict={}
disease_dict={}


#Fucntion to map entities to unique abbrieviations/values

def gen_unique_abbrev(name, abbreviation_dict):
  initials= ''.join(word[0] for word in name.split())
  abbreviation=initials
  #print('Abbreviation',abbreviation, abbreviation_dict)
  
  if row['Name'] not in abbreviation_dict: #If name not in dictionary #Condition1:  Eg: 'Mia Foster': 'MF' If twice Mia Foster keep the first one
    while abbreviation in abbreviation_dict.values(): #Condition2: if abbreviation already in dictionary value #If names Mia Foster and Mia Farhan both has suffix as MF replacesecond MF with MF1
      suffix=1
      #print('Abbreviation2',abbreviation_dict)
      while initials+str(suffix) in abbreviation_dict.values():
        suffix+=1
      abbreviation=initials + str(suffix)
    abbreviation_dict[name] = abbreviation
  else:
    abbreviation_dict[name] = abbreviation

  return abbreviation

for index, row in new_df.iterrows():
  
  name_dict[row['Name']] = gen_unique_abbrev(row['Name'], name_dict)


  if row['Address'] not in address_dict:
    if str(row['Address']).lower()!="not available":
      address_dict[row['Address']] = 'Address' + str(len(address_dict) +1)
    else:
      address_dict[row['Address']] = "Address Not Available"

  if row['Email Id'] not in email_dict:
    if str(row['Email Id']).lower()!="not available":
      email_dict[row['Email Id']] = 'Email Id' + str(len(email_dict) +1)
    else:
      email_dict[row['Email Id']] = "Email Not Available"


  if row['Phone number'] not in phone_dict:
    if str(row['Phone number']).lower()!="not available":
      phone_dict[row['Phone number']] = 'Phone' + str(len(phone_dict) +1)
    else:
      phone_dict[row['Phone number']] = "Phone Not Available"


  
  if row['Disease'] not in disease_dict: # If key/disease doesnot exist in dictionary 
   
      if str(row['Disease']).lower()!="not available":
          
          disease_dict[row['Disease']] = 'Disease' + str(len(disease_dict) +1)
      else:
          disease_dict[row['Disease']] = "Disease Not Available"

  '''
  result_dict[row['Name']] = {
    row['Name'] : name_dict[row['Name']],
    row['Address'] : address_dict[row['Address']],
    row['Email Id'] : email_dict[row['Email Id']],
    row['Phone number'] : phone_dict[row['Phone number']],
    row['Disease'] : disease_dict[row['Disease']]
  }
'''
disease_dict

{'cold': 'Disease1',
 'flu': 'Disease2',
 'allergy': 'Disease3',
 'not available': 'Disease Not Available',
 'recovering from a recent surgery': 'Disease5',
 'insomnia': 'Disease6',
 'minor injury': 'Disease7',
 'arthritis': 'Disease8'}

In [0]:
print(name_dict)
print(address_dict)
print(email_dict)
print(phone_dict)
print(disease_dict)


{'Jake Rodriguez': 'JR', 'Emma White': 'EW', 'Harper David': 'HD', 'Mia Foster': 'MF', 'Liam Baker': 'LB', 'Ava Taylor': 'AT', 'Zoe Wright': 'ZW', 'Ethan Martinez': 'EM', 'Olivia Leione': 'OL', 'Liam Thompson': 'LT', 'Harper Davis': 'HD1', 'Olivia Lee': 'OL1'}
{'789 Elm Street, Riverside': 'Address1', '505 Pine Lane, Metroville': 'Address2', '909 Oakwood Road, Lakeside': 'Address3', '111 Cedar Drive, Uptown': 'Address4', '777 Willow Lane, Downtown': 'Address5', '333 Birch Street, Suburbia': 'Address6', '909 Sycamore Avenue, Hamlet City': 'Address7', '101 Pinecrest Road, Villagetown': 'Address8', '210 Willow Lane, Cityville': 'Address9', '456 Cedar Avenue, Metroville': 'Address10', 'Not Available': 'Address Not Available'}
{'Not Available': 'Email Not Available', 'litho@example.com': 'Email Id2'}
{'(555) 123-4567': 'Phone1', 'Not Available': 'Phone Not Available', '(555) 987-6543': 'Phone3', '(555) 210-9876': 'Phone4', '(555) 876-5432': 'Phone5', '(555) 456-7890': 'Phone6', '(555) 789-0

In [0]:
new_df.columns

Index(['SlNo', 'Text', 'Response', 'Anonymized Text', 'Name', 'Address',
       'Email Id', 'Phone number', 'Disease'],
      dtype='object')

###Replace Placeholders in Anonymized Text with Entity Abbreiations

In [0]:
for index, row in new_df.iterrows():
  anno_text= row['Anonymized Text']
  name= row['Name']
  address= row['Address']
  email = row['Email Id']
  phone=row['Phone number']
  disease= row['Disease']
  #if name in name_dict:
  name_abbr= [v for k, v in name_dict.items() if name.lower() in k.lower()]
  #print(name_abbr[0])

  add_abbr= [v for k, v in address_dict.items() if address.lower() in k.lower()]
  #print(add_abbr[0])

  email_abbr= [v for k, v in email_dict.items() if email.lower() in k.lower()]
  #print(email_abbr[0])


  phone_abbr= [v for k, v in phone_dict.items() if phone.lower() in k.lower()]
  #print(phone_abbr[0])

  disease_abbr= [v for k, v in disease_dict.items() if disease.lower() in k.lower()]
  #print(disease_abbr[0])


  rep_for_anonymized_text = {"<NAME>": name_abbr[0], "<PHONE NUMBER>": phone_abbr[0], "<ADDRESS>":add_abbr[0], "<EMAIL>":email_abbr[0], "<DISEASE>": disease_abbr[0]}
  text_with_placeholders=row['Anonymized Text']
  
  new_df.at[index,'Anonymized Name'] = name_abbr[0]
  new_df.at[index,'Anonymized Phone Number'] = phone_abbr[0]
  new_df.at[index,'Anonymized Address'] = add_abbr[0]
  new_df.at[index,'Anonymized Email'] = email_abbr[0]
  new_df.at[index,'Anonymized Disease'] = disease_abbr[0]

  # use these  lines to do the replacement
  #from functools import reduce
  #reduce(lambda a, kv: a.replace(*kv), rep_for_anonymized_text.items(), str1)
  for x,y in rep_for_anonymized_text.items():
    text_with_placeholders = text_with_placeholders.replace(x, y)
  
  new_df.at[index,'Anonymized Text'] = text_with_placeholders

In [0]:
new_df.head(3)

SlNo  ...  Disease
0   1.0  ...     cold
1   2.0  ...      flu
2   3.0  ...  allergy

[3 rows x 9 columns]

In [0]:
new_df.columns

Index(['SlNo', 'Text', 'Response', 'Anonymized Text', 'Name', 'Address',
       'Email Id', 'Phone number', 'Disease', 'Anonymized Name',
       'Anonymized Phone Number', 'Anonymized Address', 'Anonymized Email',
       'Anonymized Disease'],
      dtype='object')

In [0]:
#Reorder Columns
new_df.to_csv(blob_path+'Input text entities with abbreviations.csv', columns=['SlNo','Text','Response','Anonymized Text','Name', 'Anonymized Name', 'Address', 'Anonymized Address', 'Email Id', 'Anonymized Email', 'Phone number',  'Anonymized Phone Number', 'Disease', 'Anonymized Disease'])



In [0]:
#Case2

In [0]:
system_prompt= f"Act as an AI expert. You will be given data to extract entities from it. The entities that you need to extract are given in the list ``` {enity_list} ``` . In the absence of any entity specify message Not Available.  Be precise. Do not add any information which is not present in the data. Once you extract entity - generate an approach to anonymize these entities listed in the ``` {enity_list} `` while preserving the original data's utility for analysis. The anonymization should be performed in a way that makes it difficult to reverse engineer the original entities, ensuring data privacy and confidentiality. Provide as an output the extracted entities and anonymized results. Donot include approach in output  "
#Decsribe the steps os your proposed approach for anonymizing entities
df_results_anonymized = pd.DataFrame()


In [0]:
for index, row in df.iterrows():
       
      user_prompt = f""" Anonymize entity for the below text.
                  
                  Text: { row['Text']}
                  Name Entity: <NAME>
                  Address Entity: <ADDRESS>
                  Email Id Entity: <EMAIL>
                  Phone number Entity: <PHONE NUMBER>
                  Disease Entity: <DISEASE>
                  Anonymized Text: <ANONYMIZED TEXT>
                  Anonymized Name: <ANONYMIZED NAME>
                  Anonymized Address: <ANONYMIZED ADDRESS>
                  Anonymized Email Id: <ANONYMIZED EMAIL>
                  Anonymized Phone number: <ANONYMIZED PHONE NUMBER>
                  Anonymized Disease: <ANONYMIZED DISEASE>
               
                   """
     
      response, tokens = annotater.question_answer(system_prompt, user_prompt)
      
      df_results_anonymized.at[index,'Text']= row['Text']
      df_results_anonymized.at[index,'Response']= response

In [0]:
df_results_anonymized.to_csv(blob_path+'anonymized entities.csv')

In [0]:
df_results_anonymized

Text                                           Response
0   Hi, I'm Jake Rodriguez, you can reach me at (5...  Extracted Entities:\n- Name Entity: Jake Rodri...
1   This is Emma White speaking, no calls, please....  Extracted Entities:\n- Name Entity: Emma White...
2   Hey, it's Harper David here. Call me on (555) ...  Extracted Entities:\n- Name Entity: Harper Dav...
3   Mia Foster on the line, reachable at (555) 210...  Extracted Entities:\n- Name Entity: Mia Foster...
4   Liam Baker, available at (555) 876-5432. I'm a...  Extracted Entities:\n- Name Entity: Liam Baker...
5   Hello, it's Ava Taylor. Drop me a message at (...  Extracted Entities:\n- Name Entity: Ava Taylor...
6   Zoe Wright here, no calls for now. I'm at 909 ...  Extracted Entities:\n- Name Entity: Zoe Wright...
7   This is Ethan Martinez, reach me at (555) 789-...  Extracted Entities:\n- Name Entity: Ethan Mart...
8   Hey, it's Olivia Leione. Contact me at (555) 2...  Extracted Entities:\n- Name Entity: Olivia Lei...
9   Liam Thompson checking in, (555) 321-0987 is t...  Extracted Entities:\n- Name Entity: Liam Thomp...
10  Hey, it's Jake Rodriguez, call me at (555) 123...  Extracted Entities:\n- Name Entity: Jake Rodri...
11  Emma White on the line, reachable at (555) 210...  Extracted Entities:\n- Name Entity: Emma White...
12  This is Harper Davis, drop a message at (555) ...  Extracted Entities:\n- Name Entity: Harper Dav...
13  Mia Foster here, available at (555) 876-5432. ...  Extracted Entities:\n- Name Entity: Mia Foster...
14  Liam Baker in the line, contact me at (555) 45...  Extracted Entities:\n- Name Entity: Liam Baker...
15  This is Ethan Martinez, My card number is 5555...  Extracted Entities:\n- Name Entity: Ethan Mart...
16  Hey, it's Olivia Lee. My account ID is OL98754...  Extracted Entities:\n- Name Entity: Olivia Lee...
17  Liam Thompson speakin.  My email address is li...  Extracted Entities:\n- Name Entity: Liam Thomp...
18  Hey, it's Jake Rodriguez, call me at (555) 123...  Extracted Entities:\n- Name Entity: Jake Rodri...
19  Emma White on the line, reachable at (555) 210...  Extracted Entities:\n- Name Entity: Emma White...